In [1]:
import cv2
import numpy as np
import glob
import os
import numpy as np
import matplotlib.pyplot as plt
from patchify import patchify
import tifffile as tiff
from PIL import Image
import tensorflow as tf
from tensorflow import keras
#import segmentation_models as sm
from tensorflow.keras.metrics import MeanIoU
import random
import splitfolders

In [2]:
class MetricHistory(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.iou_scores = []
        self.val_losses = []
        self.val_iou_scores = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.iou_scores.append(logs.get('iou_score'))
        self.val_losses.append(logs.get('val_loss'))
        self.val_iou_scores.append(logs.get('val_iou_score'))

In [3]:
metric_history = MetricHistory()

In [2]:
#! wget https://landcover.ai/download/landcover.ai.v1.zip

In [5]:
root_directory = '/caldera/projects/usgs/eros/users/rfleckenstein/data'
patch_size = 512

In [4]:
#Read images from repsective 'images' subdirectory
#As all images are of different size we have 2 options, either resize or crop
#But, some images are too large and some small. Resizing will change the size of real objects.
#Therefore, we will crop them to a nearest size divisible by 256 and then 
#divide all images into patches of 256x256x3. 
img_dir=root_directory+"/images"
for path, subdirs, files in os.walk(img_dir):
    #print(path)  
    dirname = path.split(os.path.sep)[-1]
    #print(dirname)
    images = os.listdir(path)  #List of all image names in this subdirectory
    #print(images)
    for i, image_name in enumerate(images):  
        if image_name.endswith(".tif"):
            #print(image_name)
            image = cv2.imread(path+"/"+image_name, 1)  #Read each image as BGR
            SIZE_X = (image.shape[1]//patch_size)*patch_size #Nearest size divisible by our patch size
            SIZE_Y = (image.shape[0]//patch_size)*patch_size #Nearest size divisible by our patch size
            image = Image.fromarray(image)
            image = image.crop((0 ,0, SIZE_X, SIZE_Y))  #Crop from top left corner
            #image = image.resize((SIZE_X, SIZE_Y))  #Try not to resize for semantic segmentation
            image = np.array(image)             

            #Extract patches from each image
            #print("Now patchifying image:", path + "/" + image_name)
            patches_img = patchify(image, (patch_size, patch_size, 3), step=patch_size)  

            for i in range(patches_img.shape[0]):
                for j in range(patches_img.shape[1]):

                    single_patch_img = patches_img[i,j,:,:]
                    single_patch_img = single_patch_img[0] #Drop the extra unecessary dimension that patchify adds.                               

                    cv2.imwrite('/caldera/projects/usgs/eros/users/rfleckenstein/512_patches/images'+
                            image_name[:-4] + "patch_" + str(i) + str(j) + ".tif", single_patch_img)
                    #image_dataset.append(single_patch_img)


In [1]:
#Now do the same as above for masks
#For this specific dataset we could have added masks to the above code as masks have extension png
mask_dir=root_directory+"/masks"
for path, subdirs, files in os.walk(mask_dir):
    #print(path)  
    dirname = path.split(os.path.sep)[-1]

    masks = os.listdir(path)  #List of all image names in this subdirectory
    for i, mask_name in enumerate(masks):  
        if mask_name.endswith(".tif"):           
            mask = cv2.imread(path + "/" + mask_name, 0)  #Read each image as Grey (or color but remember to map each color to an integer)
            SIZE_X = (mask.shape[1]//patch_size)*patch_size #Nearest size divisible by our patch size
            SIZE_Y = (mask.shape[0]//patch_size)*patch_size #Nearest size divisible by our patch size
            mask = Image.fromarray(mask)
            mask = mask.crop((0 ,0, SIZE_X, SIZE_Y))  #Crop from top left corner
            #mask = mask.resize((SIZE_X, SIZE_Y))  #Try not to resize for semantic segmentation
            mask = np.array(mask)             

            #Extract patches from each image
            #print("Now patchifying mask:", path+"/"+ mask_name)
            patches_mask = patchify(mask, (patch_size, patch_size), step=patch_size)  

            for i in range(patches_mask.shape[0]):
                for j in range(patches_mask.shape[1]):
                    single_patch_mask = patches_mask[i,j,:,:]
                    #single_patch_mask = (single_patch_img.astype('float32')) / 255. #No need to scale masks, but you can do it if you want
                    single_patch_mask = single_patch_mask[0] #Drop the extra unecessary dimension that patchify adds.
                    cv2.imwrite('/caldera/projects/usgs/eros/users/rfleckenstein/data/512_patches/masks'+
                            mask_name[:-4] + "patch_" + str(i) + str(j) + ".tif", single_patch_mask)


In [2]:
train_img_dir = '/caldera/projects/usgs/eros/users/rfleckenstein/data/512_patches/images'
train_mask_dir = '/caldera/projects/usgs/eros/users/rfleckenstein/data/512_patches/masks'

img_list = os.listdir(train_img_dir)
msk_list = os.listdir(train_mask_dir)

num_images = len(os.listdir(train_img_dir))

img_num = random.randint(0, num_images-1)

img_for_plot = cv2.imread(os.path.join(train_img_dir, img_list[img_num]), 1)

img_for_plot = cv2.cvtColor(img_for_plot, cv2.COLOR_BGR2RGB)
mask_for_plot =cv2.imread(os.path.join(train_mask_dir, msk_list[img_num]), 0)

plt.figure(figsize=(12, 8))
plt.subplot(121)
plt.imshow(img_for_plot)
plt.title('Image')
plt.subplot(122)
plt.imshow(mask_for_plot, cmap='gray')
plt.title('Mask')
plt.show()

In [3]:
input_folder = '/caldera/projects/usgs/eros/users/rfleckenstein/data/512_patches'
output_folder = '/caldera/projects/usgs/eros/users/rfleckenstein/data/train_val_data'
# Split with a ratio.
# To only split into training and validation set, set a tuple to `ratio`, i.e, `(.8, .2)`.
splitfolders.ratio(input_folder, output=output_folder, seed=42, ratio=(.80, .20), group_prefix=None) # default values

# Debugging

In [10]:
# img_dir=root_directory+"/images"

In [11]:
# image = img_dir + '/'+ os.listdir(img_dir)[0]

In [12]:
# image

In [13]:
# mask_dir=root_directory+"/masks"

In [14]:
# mask = mask_dir + '/'+os.listdir(mask_dir)[0]

In [15]:
# mask

In [16]:
# img_for_plot = cv2.imread(image, 1)

# img_for_plot = cv2.cvtColor(img_for_plot, cv2.COLOR_BGR2RGB)
# mask_for_plot =cv2.imread(mask, 0)

# plt.figure(figsize=(12, 8))
# plt.subplot(121)
# plt.imshow(img_for_plot)
# plt.title('Image')
# plt.subplot(122)
# plt.imshow(mask_for_plot, cmap='gray')
# plt.title('Mask')
# plt.show()

In [17]:
# patch_size = 256

In [18]:
# image = cv2.imread(image, 1)  #Read each image as BGR
# SIZE_X = (image.shape[1]//patch_size)*patch_size #Nearest size divisible by our patch size
# SIZE_Y = (image.shape[0]//patch_size)*patch_size #Nearest size divisible by our patch size
# image = Image.fromarray(image)
# image = image.crop((0 ,0, SIZE_X, SIZE_Y))  #Crop from top left corner
# #image = image.resize((SIZE_X, SIZE_Y))  #Try not to resize for semantic segmentation
# image = np.array(image)             

# #Extract patches from each image
# #print("Now patchifying image:", path + "/" + image_name)
# patches_img = patchify(image, (256, 256, 3), step=256)  #Step=256 for 256 patches means no overlap

# for i in range(patches_img.shape[0]):
#     for j in range(patches_img.shape[1]):

#         single_patch_img = patches_img[i,j,:,:]
#         single_patch_img = single_patch_img[0] #Drop the extra unecessary dimension that patchify adds.                               

#         cv2.imwrite('/home/jovyan/opt/Image_Seg_practice/data/testing/images/'+ "patch_" + str(i) + str(j) + ".tif", single_patch_img)


In [19]:
# mask = cv2.imread(mask, 0)  #Read each image as Grey (or color but remember to map each color to an integer)
# SIZE_X = (mask.shape[1]//patch_size)*patch_size #Nearest size divisible by our patch size
# SIZE_Y = (mask.shape[0]//patch_size)*patch_size #Nearest size divisible by our patch size
# mask = Image.fromarray(mask)
# mask = mask.crop((0 ,0, SIZE_X, SIZE_Y))  #Crop from top left corner
# #mask = mask.resize((SIZE_X, SIZE_Y))  #Try not to resize for semantic segmentation
# mask = np.array(mask)             

# #Extract patches from each image
# #print("Now patchifying mask:", path+"/"+ mask_name)
# patches_mask = patchify(mask, (256, 256), step=256)  #Step=256 for 256 patches means no overlap

# for i in range(patches_mask.shape[0]):
#     for j in range(patches_mask.shape[1]):
#         single_patch_mask = patches_mask[i,j,:,:]
#         #single_patch_img = (single_patch_img.astype('float32')) / 255. #No need to scale masks, but you can do it if you want
#         #single_patch_mask = single_patch_mask[0] #Drop the extra unecessary dimension that patchify adds.
#         cv2.imwrite('/home/jovyan/opt/Image_Seg_practice/data/testing/masks/'+ "patch_" + str(i) + str(j) + ".tif", single_patch_mask)

In [20]:
# train_img_dir = os.path.join(os.getcwd(), 'data', 'testing', 'images')
# train_mask_dir = os.path.join(os.getcwd(), 'data', 'testing', 'masks')

# img_list = os.listdir(train_img_dir)
# msk_list = os.listdir(train_mask_dir)

# num_images = len(os.listdir(train_img_dir))

# img_num = random.randint(0, num_images-1)

# img_for_plot = cv2.imread(os.path.join(train_img_dir, img_list[img_num]), 1)

# img_for_plot = cv2.cvtColor(img_for_plot, cv2.COLOR_BGR2RGB)
# mask_for_plot =cv2.imread(os.path.join(train_mask_dir, msk_list[img_num]), 0)

# plt.figure(figsize=(12, 8))
# plt.subplot(121)
# plt.imshow(img_for_plot)
# plt.title('Image')
# plt.subplot(122)
# plt.imshow(mask_for_plot, cmap='gray')
# plt.title('Mask')
# plt.show()